In [1]:
import utils.disaster_data_utils as disaster
import utils.speech_data_utils as speech

# Ensure we use only year and country pairs that both datasets share, so we get exaclty the same length feature and target matrices.

In [2]:
# Prepare training data
df_disaster = disaster.build_clean_dataframe()
df_speech = speech.build_dataframe()

C:\Users\wim_p\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:

speech_countries = set(df_speech["country"].unique())
speech_years = set(df_speech["year"].unique())
disaster_years = set(df_disaster["Year"].unique())
diaster_countries = set(df_disaster["Country"].unique())

In [4]:
# for the speech dataframes, create country year pairs
speech_country_year_pairs = []
for row in df_speech.iterrows():
    country = row[1]["country"]
    year = row[1]["year"]
    speech_country_year_pairs.append((country, year))
speech_country_year_pairs = set(speech_country_year_pairs)
    
# for the disaster dataframes, create country year pairs
disaster_country_year_pairs = []
for row in df_disaster.iterrows():
    country = row[1]["Country"]
    year = row[1]["Year"]
    disaster_country_year_pairs.append((country, year))
disaster_country_year_pairs = set(disaster_country_year_pairs)

   
   
shared_pairs = list(speech_country_year_pairs.intersection(disaster_country_year_pairs))
shared_pairs = [pair for pair in shared_pairs if pair[1] >= 1988 and pair[1] <= 2022]


# keywords = ["Co2", "climate change"]
keywords = speech.keywords

# shared_pairs = shared_pairs

In [5]:
print(len(shared_pairs))
X_train = disaster.create_feature_train_matrix(country_year_pairs=shared_pairs)
print(X_train.shape)



2056
Creating feature matrix for 2056 country-year pairs)


C:\Users\wim_p\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


(2056, 4)


In [6]:
T_train = speech.create_target_value_matrix(country_year_pairs=shared_pairs, keywords=keywords)


Creating feature matrix for 2056 country-year pairs)
prepoccesing df
adding keyword counts column to df by matching keywords in stemmed stemmed text
adding year score column to df


In [7]:
T_train

array([[0.88888889],
       [0.07228916],
       [0.62608696],
       ...,
       [0.3908046 ],
       [0.3559322 ],
       [0.10091743]])

In [8]:
print(T_train.shape)
print(X_train.shape)
# # FIXME: Should be same size.
# assert(X_train.shape[0] == T_train[0])



(2056, 1)
(2056, 4)


In [9]:
# from sklearn.linear_model import LinearRegression
# # TODO: fix Nan Values
# model.fit(X_train, T_train)

# Cross Fold Validation

In [144]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
model = LinearRegression(fit_intercept=True)


import numpy as np

def kfold_cross_validation(X, Y):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

    # Create grid of parameters to test through cross-validation
    param_grid = { 
                  'polynomialfeatures__degree': np.arange(1,3),
                 'linearregression__fit_intercept': [True, False],
                 'polynomialfeatures__include_bias':[True, False], 
                 'linearregression__positive':[True, False]
                 }


    
    model = make_pipeline(PolynomialFeatures(), LinearRegression())
    
    

    grid = GridSearchCV(model, param_grid, cv=100)
    grid.fit(X_train, Y_train)
    print(model.get_params().keys())
    # Let us know check the results with the best estimator after Grid Search
    print("Best hyperparameters found:")
    print(grid.best_params_)
    y_pred = grid.best_estimator_.predict(X_test)
    print("")

    # Compute test error and variance score
    print("Model accuracy:")
    print("Mean squared error: %.5f" % mean_squared_error(Y_test, y_pred))
    print('r2 score: %.5f' % r2_score(Y_test, y_pred))
    
    print("")

    #Plot with test data and fitted model
    # print("Plot:")
    # x.plot.scatter(x="Freedom to make life choices", y="Perceptions of corruption",c='black')
    # xfit = np.linspace(min(X_test), max(X_test), 1000)
    # yfit = grid.best_estimator_.predict(xfit[:, np.newaxis])
    # plt.plot(xfit, yfit, c='red');
    return grid

In [145]:
grid = kfold_cross_validation(X_train, T_train)

dict_keys(['memory', 'steps', 'verbose', 'polynomialfeatures', 'linearregression', 'polynomialfeatures__degree', 'polynomialfeatures__include_bias', 'polynomialfeatures__interaction_only', 'polynomialfeatures__order', 'linearregression__copy_X', 'linearregression__fit_intercept', 'linearregression__n_jobs', 'linearregression__normalize', 'linearregression__positive'])
Best hyperparameters found:
{'linearregression__fit_intercept': True, 'linearregression__positive': True, 'polynomialfeatures__degree': 1, 'polynomialfeatures__include_bias': True}

Model accuracy:
Mean squared error: 0.03783
r2 score: -0.00429



In [24]:
grid.cv_results_


{'mean_fit_time': array([0.00108573, 0.00200522, 0.00335608, 0.01082702, 0.03637023]),
 'std_fit_time': array([0.00229527, 0.00117453, 0.00443402, 0.00709385, 0.00699289]),
 'mean_score_time': array([0.00010166, 0.00043402, 0.00075557, 0.00156291, 0.00156271]),
 'std_score_time': array([0.00030499, 0.00045111, 0.00195619, 0.00468872, 0.00468814]),
 'param_polynomialfeatures__degree': masked_array(data=[1, 2, 3, 4, 5],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'polynomialfeatures__degree': 1},
  {'polynomialfeatures__degree': 2},
  {'polynomialfeatures__degree': 3},
  {'polynomialfeatures__degree': 4},
  {'polynomialfeatures__degree': 5}],
 'split0_test_score': array([-1.55716897e-02, -6.36067536e-02, -2.32420718e+02, -7.52388136e+05,
        -3.11791514e+08]),
 'split1_test_score': array([ 3.01481598e-04, -8.73733549e-04, -1.66531750e-01, -3.20040367e+01,
        -2.41494941e+02]),
 'split2_test_score': arra

In [129]:

print(X_train)

[[  2.          58.          29.          47.        ]
 [  2.          86.          43.          69.        ]
 [  1.          10.          10.          10.        ]
 ...
 [  1.           7.           7.           7.        ]
 [  2.           4.           2.           3.        ]
 [  3.         103.          34.33333333  90.        ]]


In [13]:
# map iso code to countries
import pandas as pd
path = "../../data/CO2emissions.csv"
# create df
df = pd.read_csv(path)# Replace 'your_file.csv' with the actual path to your CSV file

# create dictionary with iso codes as keys and countries as values using Country and ISO 3166-1 alpha-3 columns
d = dict(zip(df['ISO 3166-1 alpha-3'], df["Country"]))

In [14]:
d

{'AFG': 'Afghanistan',
 'ALB': 'Albania',
 'DZA': 'Algeria',
 'AND': 'Andorra',
 'AGO': 'Angola',
 'AIA': 'Anguilla',
 'ATA': 'Antarctica',
 'ATG': 'Antigua and Barbuda',
 'ARG': 'Argentina',
 'ARM': 'Armenia',
 'ABW': 'Aruba',
 'AUS': 'Australia',
 'AUT': 'Austria',
 'AZE': 'Azerbaijan',
 'BHS': 'Bahamas',
 'BHR': 'Bahrain',
 'BGD': 'Bangladesh',
 'BRB': 'Barbados',
 'BLR': 'Belarus',
 'BEL': 'Belgium',
 'BLZ': 'Belize',
 'BEN': 'Benin',
 'BMU': 'Bermuda',
 'BTN': 'Bhutan',
 'BES': 'Bonaire, Saint Eustatius and Saba',
 'BIH': 'Bosnia and Herzegovina',
 'BWA': 'Botswana',
 'BRA': 'Brazil',
 'VGB': 'British Virgin Islands',
 'BRN': 'Brunei Darussalam',
 'BGR': 'Bulgaria',
 'BFA': 'Burkina Faso',
 'BDI': 'Burundi',
 'KHM': 'Cambodia',
 'CAN': 'Canada',
 'CPV': 'Cape Verde',
 'CAF': 'Central African Republic',
 'TCD': 'Chad',
 'CHL': 'Chile',
 'CHN': 'China',
 'CXR': 'Christmas Island',
 'COL': 'Colombia',
 'COM': 'Comoros',
 'COG': 'Congo',
 'COK': 'Cook Islands',
 'CRI': 'Costa Rica',
 